#**Final Project in 37225401 Course (2020B):**
#***COVID-19: Analyzing Crime Data Before and During the Pandemic in Several US Cities***
##Submitted by: **Ricky Shama**
##Date of Submission: Aug 29, 2020
##Link to Code (Read-only): [Google Colab](https://colab.research.google.com/drive/19oW7Ix9NH3BRSGSgExwHJDZhrjwb-fF6?usp=sharing)

The aim of this project is analyzing crime data (crimes, calls for service/ 911/ PD, rate of murders, etc.) of several US cities (Chicago, Los Angeles, New York City, and Seattle), and see how the COVID-19 pandemic related measures ("Stay-at-Home" orders, social limitations, phased openings, etc.) are reflected in these datasets, and compared to 2019.
<br>
* Note #1: I followed the Stay-at-Home statewide orders' list in [*Littler*](https://www.littler.com/publication-press/publication/stay-top-stay-home-list-statewide), and offical cities' websites.
* Note #2: [The killing of George Floyd](https://www.nytimes.com/2020/05/31/us/george-floyd-investigation.html), which occured on May 25, triggered protests and riots across the country a couple of days later. I'll show how the data is affected by this.
* Note #3: You can see the charts/ maps of every city under \<City\> chapter -> Charts/ Map Section.


#Preparation

##Installations

In [ ]:
!pip install folium --upgrade

In [ ]:
!pip install turicreate

##Imports

In [3]:
from plotly.subplots import make_subplots

import datetime as dt
import folium
import folium.plugins as plugins
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import turicreate as tc
import turicreate.aggregate as agg

In [4]:
!mkdir ./datasets

In [5]:
loc = {'chi': [41.881832, -87.623177], 'la': [34.052235, -118.243683], 
       'ny': [40.730610, -73.935242], 'sea': [47.608013, -122.335167]}

#Chicago

In [6]:
!mkdir ./datasets/chi

##Downloading Relevant Datasets

###Crimes - 2019

In [ ]:
!wget https://data.cityofchicago.org/api/views/w98m-zvie/rows.csv?accessType=DOWNLOAD -O ./datasets/chi/cd_2019.csv

###Crimes - 2020

In [ ]:
!wget https://data.cityofchicago.org/api/views/qzdf-xmn8/rows.csv?accessType=DOWNLOAD -O ./datasets/chi/cd_2020.csv

##Datasets' Preparation + Preprocessing

In [ ]:
sf_cd_19 = tc.SFrame.read_csv("./datasets/chi/cd_2019.csv")
sf_cd_19

In [ ]:
sf_cd_20 = tc.SFrame.read_csv("./datasets/chi/cd_2020.csv")
sf_cd_20

In [ ]:
sf_cd = sf_cd_19.append(sf_cd_20)
sf_cd

In [ ]:
sf_cd = sf_cd.select_columns(['Date', 'Primary Type', 'Description', 
                              'Location Description', 'Arrest', 
                              'Domestic', 'Year', 'Latitude', 'Longitude'])
sf_cd

In [ ]:
sf_cd['daystring'] = sf_cd['Date'].apply(lambda x: x.split()[0])
sf_cd

In [14]:
sorted(list(sf_cd['Primary Type'].unique()))

['ARSON',
 'ASSAULT',
 'BATTERY',
 'BURGLARY',
 'CONCEALED CARRY LICENSE VIOLATION',
 'CRIM SEXUAL ASSAULT',
 'CRIMINAL DAMAGE',
 'CRIMINAL SEXUAL ASSAULT',
 'CRIMINAL TRESPASS',
 'DECEPTIVE PRACTICE',
 'GAMBLING',
 'HOMICIDE',
 'HUMAN TRAFFICKING',
 'INTERFERENCE WITH PUBLIC OFFICER',
 'INTIMIDATION',
 'KIDNAPPING',
 'LIQUOR LAW VIOLATION',
 'MOTOR VEHICLE THEFT',
 'NARCOTICS',
 'NON-CRIMINAL',
 'OBSCENITY',
 'OFFENSE INVOLVING CHILDREN',
 'OTHER NARCOTIC VIOLATION',
 'OTHER OFFENSE',
 'PROSTITUTION',
 'PUBLIC INDECENCY',
 'PUBLIC PEACE VIOLATION',
 'ROBBERY',
 'SEX OFFENSE',
 'STALKING',
 'THEFT',
 'WEAPONS VIOLATION']

The following dataset includes every crime case in Chicago since January 1, 2019. I chose to focus on murders, domestic violence cases, drug cases and cases involving children.

In [15]:
sf_cd['domestic'] = sf_cd['Domestic'].apply(lambda x: 1 if (x =='true') else 0)
sf_cd['murder'] = sf_cd['Primary Type'].apply(lambda x: 1 if (x =='HOMICIDE') else 0)
sf_cd['drugs'] = sf_cd['Primary Type'].apply(lambda x: 1 if ((x =='NARCOTICS') | 
                                                             (x =='OTHER NARCOTIC VIOLATION')) else 0)
sf_cd['childern'] = sf_cd['Primary Type'].apply(lambda x: 1 if (x =='OFFENSE INVOLVING CHILDREN') else 0)
sf_cd

Date,Primary Type,Description,Location Description,Arrest,Domestic
09/04/2019 05:00:00 AM,NARCOTICS,POSSESS - METHAMPHETAMINE,RESIDENCE,false,false
09/27/2019 11:14:00 AM,NARCOTICS,MANUFACTURE / DELIVER -HEROIN (WHITE) ...,VEHICLE NON-COMMERCIAL,false,false
09/16/2019 12:01:00 AM,DECEPTIVE PRACTICE,EMBEZZLEMENT,MEDICAL / DENTAL OFFICE,false,false
08/24/2019 11:15:00 AM,NARCOTICS,MANUFACTURE / DELIVER -HEROIN (WHITE) ...,SIDEWALK,false,false
08/23/2019 12:45:00 PM,NARCOTICS,MANUFACTURE / DELIVER -HEROIN (WHITE) ...,SIDEWALK,false,false
09/14/2019 10:32:00 AM,NARCOTICS,MANUFACTURE / DELIVER -HEROIN (WHITE) ...,VEHICLE NON-COMMERCIAL,false,false
08/03/2019 12:00:00 AM,OTHER OFFENSE,HARASSMENT BY ELECTRONICMEANS ...,RESIDENCE,false,true
09/10/2019 02:04:00 PM,NARCOTICS,MANUFACTURE / DELIVER -CRACK ...,SIDEWALK,false,false
09/18/2019 12:32:00 PM,NARCOTICS,POSSESS - COCAINE,APARTMENT,false,false
09/05/2019 01:00:00 PM,NARCOTICS,POSSESS - CRACK,SIDEWALK,false,false


In [ ]:
sf_cd_counts = sf_cd.groupby(key_column_names='daystring', operations={'tot_count': agg.COUNT(),
                                                                       'domestic_count': agg.SUM('domestic'),
                                                                       'murder_count': agg.SUM('murder'),
                                                                       'drugs_count': agg.SUM('drugs'),
                                                                       'childern_count': agg.SUM('childern')})
sf_cd_counts['domestic_rate'] = sf_cd_counts.apply(lambda x: x['domestic_count'] / x['tot_count'] * 100)
sf_cd_counts['drugs_rate'] = sf_cd_counts.apply(lambda x: x['drugs_count'] / x['tot_count'] * 100)
sf_cd_counts['monthstring'] = sf_cd_counts.apply(lambda x: x['daystring'].split('/')[0] + 
                                                 '/' + x['daystring'].split('/')[2])

sf_cd_counts

In [ ]:
sf_cd_counts_month = sf_cd_counts.groupby(key_column_names='monthstring', 
                                          operations={'tot_count': agg.SUM('tot_count'),
                                                      'murder_count': agg.SUM('murder_count'),
                                                      'childern_count': agg.SUM('childern_count')})
sf_cd_counts_month['murder_rate'] = sf_cd_counts_month.apply(lambda x: x['murder_count'] / x['tot_count'] * 100)
sf_cd_counts_month['children_rate'] = sf_cd_counts_month.apply(lambda x: x['childern_count'] / x['tot_count'] * 100)

sf_cd_counts_month

The following dataset summarizes the number of total cases and the cases' types I chose to focus on, and their rate [%] (computed as: # specfic case type / # total cases * 100) for each **day**.  

In [18]:
df_cd_counts = sf_cd_counts.to_dataframe()
df_cd_counts['date'] = pd.to_datetime(df_cd_counts['daystring'])
df_cd_counts.sort_values(by=['date'], inplace=True, ascending=True)
df_cd_counts

,daystring,tot_count,domestic_count,murder_count,drugs_count,childern_count,domestic_rate,drugs_rate,monthstring,date
72,01/01/2019,1018,220,0,36,41,21.611002,3.536346,01/2019,2019-01-01
48,01/02/2019,652,106,0,49,0,16.257669,7.515337,01/2019,2019-01-02
113,01/03/2019,751,122,2,53,2,16.245007,7.057257,01/2019,2019-01-03
150,01/04/2019,767,106,0,68,3,13.820078,8.865711,01/2019,2019-01-04
416,01/05/2019,790,156,1,75,6,19.746835,9.493671,01/2019,2019-01-05
...,...,...,...,...,...,...,...,...,...,...
7,08/17/2020,604,114,1,18,4,18.874172,2.980132,08/2020,2020-08-17
82,08/18/2020,589,121,3,15,6,20.543294,2.546689,08/2020,2020-08-18
288,08/19/2020,574,108,0,7,3,18.815331,1.219512,08/2020,2020-08-19
258,08/20/2020,596,126,2,11,7,21.140940,1.845638,08/2020,2020-08-20


The following dataset summarizes the number of total cases and other cases' types I chose to focus on, and their rate [%] for each **month**.

In [19]:
df_cd_counts_month = sf_cd_counts_month.to_dataframe()
df_cd_counts_month['month'] = pd.to_datetime(df_cd_counts_month['monthstring'])
df_cd_counts_month['month'] = df_cd_counts_month['month'].apply(lambda x: x.strftime('%Y-%m'))
df_cd_counts_month.sort_values(by=['month'], inplace=True, ascending=True)
df_cd_counts_month

,monthstring,tot_count,murder_count,childern_count,murder_rate,children_rate,month
12,01/2019,19612,23,190,0.117275,0.968795,2019-01
14,02/2019,18373,25,187,0.136069,1.017798,2019-02
18,03/2019,20389,36,198,0.176566,0.971112,2019-03
19,04/2019,20988,60,245,0.285878,1.167334,2019-04
8,05/2019,23608,55,243,0.232972,1.029312,2019-05
7,06/2019,23545,52,173,0.220854,0.734763,2019-06
5,07/2019,24788,44,170,0.177505,0.685816,2019-07
1,08/2019,24303,51,179,0.209851,0.736535,2019-08
9,09/2019,22333,51,184,0.228362,0.823893,2019-09
11,10/2019,21567,40,187,0.185469,0.867065,2019-10


##Charts 

###Daily Charts (2019-Present)

* The Illinois' "Stay-at-Home" order started on March 21, and ended on May 30.
* In the following chart, we can see some decrease in the number of crimes cases since the end of March, and a sharp increase on May 31, following the killing of George Floyd.

In [20]:
fig = px.bar(df_cd_counts, x="date", y="tot_count",
             color='tot_count',
             title='Daily # Crimes in Chicago (2019-Present)', 
             labels={"date": "Date",
                     "tot_count": "#"})
fig.show()

* The following chart shows that since the end of March, the daily rate of domestic violence has **increased** (compared to 2019).

In [21]:
fig = px.line(df_cd_counts, x='date', y='domestic_rate',
              title='Daily Rate of Domestic Crimes in Chicago (2019-Present)', 
              labels={'date': 'Date',
                      'domestic_rate': '%'})
fig.show()

* The following chart shows that since the end of March, the daily rate of drugs' offenses has **decreased** (compared to 2019).

In [170]:
fig = px.line(df_cd_counts, x='date', y='drugs_rate',
              title='Daily Rate of Drugs\' Offenses in Chicago (2019-Present)', 
              labels={'date': 'Date',
                      'drugs_rate': '%'})
fig['data'][0]['line']['color']="#800000"
fig.show()

###Monthly Charts (2019-Present)

* The following chart shows that since the end of March, the monthly rate of murders has **increased sharply** (and even doubled itself compared to 2019).

In [23]:
fig = px.bar(df_cd_counts_month, x="month", y="murder_rate",
             color='murder_rate',
             title='Monthly Rate of Murders in Chicago (2019-Present)', 
             labels={"month": "Month",
                     "murder_rate": "%"})
fig.show()

* The following chart shows that since the end of March, there is a slight decrease in the rate of children-involved crimes, but it is higher compared to 2019.

In [24]:
fig = px.bar(df_cd_counts_month, x="month", y="children_rate",
             color='children_rate',
             title='Monthly Rate of Children-Involved Crimes in Chicago (2019-Present)', 
             labels={"month": "Month",
                     "children_rate": "%"})
fig.show()

##Daily Crime Map (2019-Present)

* An Interactive map of every crime reported in Chicago since January 1, 2019 (it displays, for example, the sharp increase in the number of crimes on May 31, 2020).

In [25]:
days = list(df_cd_counts['daystring'])
data_cd = []
for day in days:
  l_temp = []
  sf_temp = sf_cd[sf_cd['daystring'] == day]
  for row in range(sf_temp.num_rows()):
    if (sf_temp['Latitude'][row] is not None) and (sf_temp['Longitude'][row] is not None):
      l_temp.append([sf_temp['Latitude'][row], sf_temp['Longitude'][row]])
  data_cd.append(l_temp)

In [ ]:
m = folium.Map(loc['chi'], zoom_start=10)
m

In [27]:
hm = plugins.HeatMapWithTime(
    data=data_cd,
    index=days,
    auto_play=True,
)

hm.add_to(m)

m

Output hidden; open in https://colab.research.google.com to view.

#Los Angeles

In [28]:
!mkdir ./datasets/la

##Downloading Relevant Datasets

###Crime Data 2010-2019

In [ ]:
!wget https://data.lacity.org/api/views/63jg-8b9z/rows.csv?accessType=DOWNLOAD -O ./datasets/la/cd_2010_2019.csv

###Crime Data 2020

In [ ]:
!wget https://data.lacity.org/api/views/2nrs-mtv8/rows.csv?accessType=DOWNLOAD -O ./datasets/la/cd_2020.csv

###LAPD Calls for Service 2019

In [ ]:
!wget https://data.lacity.org/api/views/r4ka-x5je/rows.csv?accessType=DOWNLOAD -O ./datasets/la/cfs_2019.csv

###LAPD Calls for Service 2020

In [ ]:
!wget https://data.lacity.org/api/views/84iq-i2r6/rows.csv?accessType=DOWNLOAD -O ./datasets/la/cfs_2020.csv

##Datasets' Preparation + Preprocessing

In [ ]:
sf_cd_19 = tc.SFrame.read_csv("./datasets/la/cd_2010_2019.csv")
sf_cd_19

In [ ]:
sf_cd_19['Year'] = sf_cd_19['DATE OCC'].apply(lambda x: x.split()[0].split("/")[2])
sf_cd_19 = sf_cd_19[sf_cd_19['Year'] == '2019']
sf_cd_19

In [ ]:
sf_cd_19 = sf_cd_19.select_columns(['DATE OCC', 'TIME OCC', 'AREA NAME', 
                                    'Crm Cd Desc', 'Vict Age', 'Vict Sex', 
                                    'Vict Descent', 'Premis Desc', 'Weapon Used Cd', 
                                    'Weapon Desc', 'Status Desc', 'LAT', 'LON', 
                                    'Year'])
sf_cd_19

In [ ]:
sf_cd_20 = tc.SFrame.read_csv("./datasets/la/cd_2020.csv")
sf_cd_20

In [ ]:
sf_cd_20['Year'] = '2020'
sf_cd_20

In [ ]:
sf_cd_20 = sf_cd_20.select_columns(['DATE OCC', 'TIME OCC', 'AREA NAME', 
                                    'Crm Cd Desc', 'Vict Age', 'Vict Sex', 
                                    'Vict Descent', 'Premis Desc', 'Weapon Used Cd', 
                                    'Weapon Desc', 'Status Desc', 'LAT', 'LON', 
                                    'Year'])
sf_cd_20

In [ ]:
sf_cd_19_20 = sf_cd_19.append(sf_cd_20)
sf_cd_19_20

In [ ]:
sf_cd_19_20['daystring'] = sf_cd_19_20['DATE OCC'].apply(lambda x: x.split()[0])
sf_cd_19_20

In [41]:
sorted(list(sf_cd_19_20['Crm Cd Desc'].unique()))

['ABORTION/ILLEGAL',
 'ARSON',
 'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER',
 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
 'ATTEMPTED ROBBERY',
 'BATTERY - SIMPLE ASSAULT',
 'BATTERY ON A FIREFIGHTER',
 'BATTERY POLICE (SIMPLE)',
 'BATTERY WITH SEXUAL CONTACT',
 'BEASTIALITY, CRIME AGAINST NATURE SEXUAL ASSLT WITH ANIM',
 'BIGAMY',
 'BIKE - ATTEMPTED STOLEN',
 'BIKE - STOLEN',
 'BOAT - STOLEN',
 'BOMB SCARE',
 'BRANDISH WEAPON',
 'BRIBERY',
 'BUNCO, ATTEMPT',
 'BUNCO, GRAND THEFT',
 'BUNCO, PETTY THEFT',
 'BURGLARY',
 'BURGLARY FROM VEHICLE',
 'BURGLARY FROM VEHICLE, ATTEMPTED',
 'BURGLARY, ATTEMPTED',
 'CHILD ABANDONMENT',
 'CHILD ABUSE (PHYSICAL) - AGGRAVATED ASSAULT',
 'CHILD ABUSE (PHYSICAL) - SIMPLE ASSAULT',
 'CHILD ANNOYING (17YRS & UNDER)',
 'CHILD NEGLECT (SEE 300 W.I.C.)',
 'CHILD PORNOGRAPHY',
 'CHILD STEALING',
 'CONSPIRACY',
 'CONTEMPT OF COURT',
 'CONTRIBUTING',
 'COUNTERFEIT',
 'CREDIT CARDS, FRAUD USE ($950 & UNDER',
 'CREDIT CARDS, FRAUD USE ($950.01 & OVER)',

In [42]:
vaw_acts = ['ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
            'BATTERY - SIMPLE ASSAULT',
            'BATTERY WITH SEXUAL CONTACT',
            'CRIMINAL HOMICIDE', 'CRIMINAL THREATS - NO WEAPON DISPLAYED', 
            'INTIMATE PARTNER - AGGRAVATED ASSAULT', 
            'INTIMATE PARTNER - SIMPLE ASSAULT', 
            'RAPE, ATTEMPTED', 'RAPE, FORCIBLE', 'STALKING', 
            'THREATENING PHONE CALLS/LETTERS']

* The following dataset includes every crime case in LA since January 1, 2019. I chose to focus on murders, cruelty to animals, child victims, and I also tried to examine violence against women cases (by filtering relevant crime-desciption's key-words and female victims).

In [43]:
sf_cd_19_20['child_vict'] = sf_cd_19_20['Vict Age'].apply(lambda x: 1 if ((x > 0) and (x <=13)) else 0)
sf_cd_19_20['cruelty_to_animal'] = sf_cd_19_20['Crm Cd Desc'].apply(lambda x: 1 if (x =='CRUELTY TO ANIMALS') else 0)
sf_cd_19_20['murder'] = sf_cd_19_20['Crm Cd Desc'].apply(lambda x: 1 if (x =='CRIMINAL HOMICIDE') else 0)
sf_cd_19_20['vaw_vict'] = sf_cd_19_20.apply(lambda x: 1 if (x['Crm Cd Desc'] in vaw_acts) and 
                                            (x['Vict Sex'] == 'F') else 0)
sf_cd_19_20

DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent
03/08/2019 12:00:00 AM,1500,Mission,VEHICLE - STOLEN,0,,
05/30/2019 12:00:00 AM,2100,Olympic,VEHICLE - STOLEN,0,,
08/06/2019 12:00:00 AM,800,Newton,THEFT FROM MOTOR VEHICLE- PETTY ($950 & UNDER) ...,0,,
05/26/2019 12:00:00 AM,100,Harbor,VEHICLE - STOLEN,0,,
11/01/2019 12:00:00 AM,2230,Wilshire,VEHICLE - STOLEN,0,,
11/07/2019 12:00:00 AM,2200,Foothill,VEHICLE - STOLEN,0,,
06/29/2019 12:00:00 AM,1300,Foothill,VEHICLE - STOLEN,0,,
01/28/2019 12:00:00 AM,2120,Wilshire,VEHICLE - STOLEN,0,,
09/18/2019 12:00:00 AM,430,Central,BURGLARY,33,M,H
01/12/2019 12:00:00 AM,1800,Olympic,VEHICLE - STOLEN,0,,


In [ ]:
sf_cd_19_20_counts = sf_cd_19_20.groupby(key_column_names='daystring', operations={'tot_count': agg.COUNT(),
                                                                                    'child_vict_count': agg.SUM('child_vict'),
                                                                                    'animal_count': agg.SUM('cruelty_to_animal'),
                                                                                    'murder_count': agg.SUM('murder'),
                                                                                    'vaw_count': agg.SUM('vaw_vict')})


sf_cd_19_20_counts['child_rate'] = sf_cd_19_20_counts.apply(lambda x: x['child_vict_count'] / x['tot_count'] * 100)
sf_cd_19_20_counts['vaw_rate'] = sf_cd_19_20_counts.apply(lambda x: x['vaw_count'] / x['tot_count'] * 100)
sf_cd_19_20_counts['monthstring'] = sf_cd_19_20_counts.apply(lambda x: x['daystring'].split('/')[0] + 
                                                 '/' + x['daystring'].split('/')[2])

sf_cd_19_20_counts

* **Daily** counts and rates of the crime-types I chose:

In [45]:
df_cd_19_20_counts = sf_cd_19_20_counts.to_dataframe()
df_cd_19_20_counts['date'] = pd.to_datetime(df_cd_19_20_counts['daystring'])
df_cd_19_20_counts.sort_values(by=['date'], inplace=True, ascending=True)
df_cd_19_20_counts

,daystring,tot_count,child_vict_count,animal_count,murder_count,vaw_count,child_rate,vaw_rate,monthstring,date
73,01/01/2019,905,69,0,0,121,7.624309,13.370166,01/2019,2019-01-01
48,01/02/2019,502,2,0,1,66,0.398406,13.147410,01/2019,2019-01-02
114,01/03/2019,563,7,1,0,59,1.243339,10.479574,01/2019,2019-01-03
151,01/04/2019,580,3,0,0,72,0.517241,12.413793,01/2019,2019-01-04
418,01/05/2019,578,5,0,0,65,0.865052,11.245675,01/2019,2019-01-05
...,...,...,...,...,...,...,...,...,...,...
259,08/20/2020,418,2,0,0,88,0.478469,21.052632,08/2020,2020-08-20
578,08/21/2020,465,3,0,1,68,0.645161,14.623656,08/2020,2020-08-21
488,08/22/2020,462,6,0,0,90,1.298701,19.480519,08/2020,2020-08-22
345,08/23/2020,369,0,0,1,81,0.000000,21.951220,08/2020,2020-08-23


In [ ]:
sf_cd_19_20_counts_month = sf_cd_19_20_counts.groupby(key_column_names='monthstring', 
                                                      operations={'tot_count': agg.SUM('tot_count'),
                                                                  'murder_count': agg.SUM('murder_count'),
                                                                  'animal_count': agg.SUM('animal_count')})
sf_cd_19_20_counts_month['murder_rate'] = sf_cd_19_20_counts_month.apply(lambda x: x['murder_count'] / x['tot_count'] * 100)
sf_cd_19_20_counts_month['animal_rate'] = sf_cd_19_20_counts_month.apply(lambda x: x['animal_count'] / x['tot_count'] * 100)

sf_cd_19_20_counts_month

* **Monthly** counts and rates of the crime-types I chose:

In [47]:
df_cd_19_20_counts_month = sf_cd_19_20_counts_month.to_dataframe()
df_cd_19_20_counts_month['month'] = pd.to_datetime(df_cd_19_20_counts_month['monthstring'])
df_cd_19_20_counts_month['month'] = df_cd_19_20_counts_month['month'].apply(lambda x: x.strftime('%Y-%m'))
df_cd_19_20_counts_month.sort_values(by=['month'], inplace=True, ascending=True)
df_cd_19_20_counts_month

,monthstring,tot_count,murder_count,animal_count,murder_rate,animal_rate,month
12,01/2019,18272,19,5,0.103984,0.027364,2019-01
14,02/2019,16189,14,6,0.086478,0.037062,2019-02
18,03/2019,18743,27,4,0.144054,0.021341,2019-03
19,04/2019,17844,22,8,0.123291,0.044833,2019-04
8,05/2019,18564,22,0,0.118509,0.000000,2019-05
7,06/2019,18251,24,5,0.131500,0.027396,2019-06
5,07/2019,18992,27,4,0.142165,0.021061,2019-07
1,08/2019,18838,21,4,0.111477,0.021234,2019-08
9,09/2019,17887,23,6,0.128585,0.033544,2019-09
11,10/2019,18067,21,4,0.116234,0.022140,2019-10


In [ ]:
sf_cfs_19 = tc.SFrame.read_csv("./datasets/la/cfs_2019.csv")
sf_cfs_19

In [ ]:
sf_cfs_20 = tc.SFrame.read_csv("./datasets/la/cfs_2020.csv")
sf_cfs_20

In [ ]:
sf_cfs = sf_cfs_19.append(sf_cfs_20)
sf_cfs

In [ ]:
sf_cfs['daystring'] = sf_cfs['Dispatch_Date'].apply(lambda x: x.split()[0])
sf_cfs['year'] = sf_cfs['Dispatch_Date'].apply(lambda x: x.split()[0].split('/')[2])

sf_cfs

In [52]:
call_types = sorted(list(sf_cfs['Call_Type_Text'].unique()))
call_types

['311/INDECENT EXPOSUR',
 '415 GROUP GANG ACT',
 '459 RECREATED FALSE',
 '484 I/P',
 'ABANDONED',
 'ABUSE',
 'ABUSE I/P',
 'ABUSE INV ESCARS',
 'ABUSE INVEST',
 'ABUSE J/O',
 'ABUSE SUSP',
 'ABUSE SUSP J/L',
 'ABUSE SUSP NOW',
 'ABUSE SUSP/VICT IN L',
 'ADULT',
 'ADULT BUS',
 'ADULT U/R',
 'ADW',
 'ADW POSS DOM VIOL',
 'AIRCRAFT',
 'ALARM',
 'ALONE',
 'ALONE INVEST',
 'AMB',
 'AMB 187',
 'AMB 187 INVEST',
 'AMB 187 J/O',
 'AMB ABUSE',
 'AMB ABUSE INVEST',
 'AMB ABUSE J/O',
 'AMB ABUSE SUSP',
 'AMB AIRCRAFT',
 'AMB ASSIST INVALID',
 'AMB ATT J/O',
 'AMB ATT SUICIDE',
 'AMB ATT SUSP',
 'AMB BUS',
 'AMB CHILD',
 'AMB CPI',
 'AMB CUTTING',
 'AMB CUTTING HATE CRI',
 'AMB CZN HLDG H&R FEL',
 'AMB D/B',
 'AMB DOM VIOL',
 'AMB DOM VIOL INVEST',
 'AMB DOM VIOL J/O',
 'AMB DOM VIOL SUSP',
 'AMB DUI',
 'AMB EXPLOSION',
 'AMB FEM',
 'AMB FIGHT',
 'AMB GTA',
 'AMB GTA J/O',
 'AMB GTA SUSP',
 'AMB H & R FEL',
 'AMB H & R FEL CPI',
 'AMB H & R FEL DUI',
 'AMB HATE CRIME INVD',
 'AMB I/P',
 'AMB INV H

In [53]:
domestic_calls = [c for c in call_types if "DOM" in c]
domestic_calls

['ADW POSS DOM VIOL',
 'AMB DOM VIOL',
 'AMB DOM VIOL INVEST',
 'AMB DOM VIOL J/O',
 'AMB DOM VIOL SUSP',
 'ATT DOM VIOL',
 'BATTERY DOMESTIC VIO',
 'CITZ HLDG DOM VIOL',
 'CZN HLDG DOM VIOL',
 'DOM VIOL',
 'DOM VIOL I/P',
 'DOM VIOL IN PROGRESS',
 'DOM VIOL INVEST',
 'DOM VIOL INVESTIGATI',
 'DOM VIOL J/O',
 'DOM VIOL R/O',
 'DOM VIOL R/O VIOLATI',
 'DOM VIOL SUSP',
 'DOM VIOL SUSP J/L',
 'DOM VIOL SUSP NOW',
 'OFCR HLDG DOM VIOL',
 'POSS AMB DOM VIOL',
 'POSS DOM VIOL',
 'POSS DOM VIOL I/P',
 'POSS DOM VIOL SUSP']

In [ ]:
sf_cfs['dom'] = sf_cfs.apply(lambda x: 1 if x['Call_Type_Text'] in domestic_calls else 0)
sf_cfs

* The following dataset includes every call-for-service as reported LA since January 1, 2019. I chose to focus on domestice violence related calls (so later we will see if the COVID-19 "Stay-at-Home" order has indeed a negative effect on domestic violence rate).

In [55]:
sf_cfs['hour'] = sf_cfs['Dispatch_Time'].apply(lambda x: x.split(':')[0])
sf_cfs

Incident_Number,Area_Occ,Rpt_Dist,Dispatch_Date,Dispatch_Time,Call_Type_Code
LPD190704000047,N Hollywood,1532,07/04/2019 12:00:00 AM,00:14:49,507F
LPD190529006334,Outside,None,05/29/2019 12:00:00 AM,21:28:53,006
LPD190218001024,Southwest,379,02/18/2019 12:00:00 AM,07:46:54,507C
PD19120600001346,Northeast,1109,12/06/2019 12:00:00 AM,08:32:27,006
LPD190216004881,Van Nuys,933,02/16/2019 12:00:00 AM,21:14:49,507P
PD19122100000089,Southwest,392,12/21/2019 12:00:00 AM,00:21:03,006
LPD190719001920,Outside,None,07/19/2019 12:00:00 AM,09:56:22,006
LPD190525000475,Topanga,2177,05/25/2019 12:00:00 AM,02:12:28,459PSN
LPD190203003749,Mission,1981,02/03/2019 12:00:00 AM,19:50:18,904AP
LPD190717001707,Mission,1983,07/17/2019 12:00:00 AM,08:56:38,006


In [ ]:
sf_cfs_counts = sf_cfs.groupby(key_column_names='daystring', operations={'tot_count': agg.COUNT(), 
                                                                         'dom_count': agg.SUM('dom')})
sf_cfs_counts['dom_rate'] = sf_cfs_counts.apply(lambda x: x['dom_count'] / x['tot_count'] * 100)
sf_cfs_counts

* **Daily** count and rate of domestic violence related calls-for-service:

In [57]:
df_cfs_counts = sf_cfs_counts.to_dataframe()
df_cfs_counts['date'] = pd.to_datetime(df_cfs_counts['daystring'])
df_cfs_counts.sort_values(by=['date'], inplace=True, ascending=True)
df_cfs_counts

,daystring,tot_count,dom_count,dom_rate,date
72,01/01/2019,4831,159,3.291244,2019-01-01
48,01/02/2019,4853,102,2.101793,2019-01-02
113,01/03/2019,5239,84,1.603359,2019-01-03
150,01/04/2019,5192,93,1.791217,2019-01-04
416,01/05/2019,4155,104,2.503008,2019-01-05
...,...,...,...,...,...
82,08/18/2020,4444,128,2.880288,2020-08-18
288,08/19/2020,4506,111,2.463382,2020-08-19
258,08/20/2020,4792,116,2.420701,2020-08-20
576,08/21/2020,4494,127,2.825990,2020-08-21


In [ ]:
sf_cfs_dom_counts_area = sf_cfs.groupby(key_column_names=['Area_Occ', 'year'], operations={'tot_count': agg.COUNT(), 
                                                                                           'dom_count': agg.SUM('dom')})
sf_cfs_dom_counts_area['dom_rate'] = sf_cfs_dom_counts_area.apply(lambda x: x['dom_count'] / x['tot_count'] * 100)

sf_cfs_dom_counts_area

* For each area in LA, this dataset includes the **Yearly** (2019, 2020 (as for now) ) count and rate of domestic violence 
related calls-for-service:

In [59]:
df_cfs_dom_counts_area = sf_cfs_dom_counts_area.to_dataframe()
df_cfs_dom_counts_area

,Area_Occ,year,tot_count,dom_count,dom_rate
0,Hollywood,2020,36123,825,2.283863
1,77th Street,2020,45328,2851,6.289711
2,Devonshire,2020,25493,768,3.012592
3,Outside,2020,465376,38,0.008165
4,Devonshire,2019,42895,1282,2.988693
5,Outside,2019,862659,94,0.010897
6,77th Street,2019,72433,4447,6.139467
7,Pacific,2019,63492,1219,1.919927
8,Mission,2019,48394,2330,4.814646
9,Newton,2019,56092,2833,5.050631


##Charts

###Daily Charts (2019-Present)

* The California's "Stay-at-Home" order started on March 19, and ended gradually starting from May 8.
* In the following chart, we can see some decrease in the number of crimes cases since the end of March, that lasted less than a month, and a sharp increase on May 29 until June 1, following the killing of George Floyd.
* But in overall, since March 2020, the number of crime cases is slightly lower compared to 2019.

In [60]:
fig = px.bar(df_cd_19_20_counts, x="date", y="tot_count",
             title='Daily # Crimes in LA (2019-Present)',
             color = "tot_count",
             labels={"date": "Date",
                     "tot_count": "#"})
fig.show()

* Since March 2020, we can see that the rate of crimes against children is slightly lower compared to 2019:

In [61]:
fig = px.line(df_cd_19_20_counts, x='date', y='child_rate',
              title='Daily Rate of Crimes Against Children (Aged 0-13) in LA (2019-Present)', 
              labels={'date': 'Date',
                      'child_rate': '%'})
fig['data'][0]['line']['color']="#008080"
fig.show()

* The violence against women is quite the same compared to 2019, although there is an increase in the last week:

In [62]:
fig = px.line(df_cd_19_20_counts, x='date', y='vaw_rate',
              title='Daily Rate of Violence Against Women in LA (2019-Present)', 
              labels={'date': 'Date',
                      'vaw_rate': '%'})
fig['data'][0]['line']['color']="#FF00FF"
fig.show()

* The rate of domestic violence calls for service was quite the same as in 2019, but since June 2020, there is an increase:

In [63]:
fig = px.line(df_cfs_counts, x='date', y='dom_rate',
              title='Daily Rate of Domestic Violence\'s Calls for Service in LA (2019-Present)', 
              labels={'date': 'Date',
                      'dom_rate': '%'})
fig['data'][0]['line']['color']="#800080"
fig.show()

###Monthly Charts (2019-Present)

* The following chart shows that since the end of March, the monthly rate of murders has **increased** (and compared to 2019).

In [64]:
fig = px.bar(df_cd_19_20_counts_month, x="month", y="murder_rate",
             color='murder_rate',
             title='Monthly Rate of Murders in LA (2019-Present)', 
             labels={"month": "Month",
                     "murder_rate": "%"})
fig.show()

* The following chart shows that since the end of March, the monthly rate of cruelty against animals has **decreased** (and compared to 2019).

In [65]:
fig = px.bar(df_cd_19_20_counts_month, x="month", y="animal_rate",
             color='animal_rate',
             title='Monthly Rate of Cruelty Against Animals in LA (2019-Present)', 
             labels={"month": "Month",
                     "animal_rate": "%"})
fig.show()

###Yearly Charts (2019 vs. 2020 (As of Today) )

* The typical hours of reporting about domestic violence cases is quite the same as in 2019:

In [67]:
x19 = list(sf_cfs[(sf_cfs['year'] == '2019') & (sf_cfs['dom'] == 1)]['hour'])
x20 = list(sf_cfs[(sf_cfs['year'] == '2020') & (sf_cfs['dom'] == 1)]['hour'])

fig = go.Figure()
fig.add_trace(go.Histogram(x=x19, name='2019'))
fig.add_trace(go.Histogram(x=x20, name='2020 (As of Today)'))

fig.update_layout(barmode='overlay', 
                  title_text='Hour-of-the-Day Histogram of Domestic Violence\'s Calls for Service in LA (2019-Present)', 
                  xaxis_title_text='Hour',
                  yaxis_title_text='Count')
fig.update_traces(opacity=0.75)
fig.show()

* The yearly domestic violence calls for service per area in LA:

In [76]:
labels_19 = df_cfs_dom_counts_area[df_cfs_dom_counts_area['year'] == '2019']['Area_Occ']
labels_20 = df_cfs_dom_counts_area[df_cfs_dom_counts_area['year'] == '2020']['Area_Occ']

values_19 = df_cfs_dom_counts_area[df_cfs_dom_counts_area['year'] == '2019']['dom_count']
values_20 = df_cfs_dom_counts_area[df_cfs_dom_counts_area['year'] == '2020']['dom_count']

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels_19, values=values_19, name="2019"), 1, 1)
fig.add_trace(go.Pie(labels=labels_20, values=values_20, name="2020"), 1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent")

fig.update_layout(
    title_text="Yearly Domestic Violence\'s Calls for Service in LA",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='2019', x=0.19, y=0.5, font_size=20, showarrow=False),
                 dict(text='2020 (As of Today)', x=0.85, y=0.5, font_size=14, showarrow=False)])
fig.show()

* In the following chart I tried to see in which area(s) in LA there was an increase/decrease, compared to 2019, in the domestic calls for service rate out of all calls for service that came from this area.

In [72]:
fig = px.bar(df_cfs_dom_counts_area, x="Area_Occ", y="dom_rate",
             color='year',
             barmode='group', 
             title="Yearly Domestic Violence\'s Calls for Service Rate Per Area in LA", 
             labels={"Area_Occ": "Area",
                     "dom_rate": "%"})
fig.show()

##Daily Crime Map (2019-Present)

* An Interactive map of every crime reported in LA since January 1, 2019 (it displays, for example, the sharp increase in the number of reported crimes at the end of May 2020).

In [73]:
days = list(df_cd_19_20_counts['daystring'])
data_cd = []
for day in days:
  l_temp = []
  sf_temp = sf_cd_19_20[sf_cd_19_20['daystring'] == day]
  for row in range(sf_temp.num_rows()):
    l_temp.append([sf_temp['LAT'][row], sf_temp['LON'][row]])
  data_cd.append(l_temp)

In [ ]:
m = folium.Map(loc['la'], zoom_start=11)
m

In [75]:
hm = plugins.HeatMapWithTime(
    data=data_cd,
    index=days,
    auto_play=True,
)

hm.add_to(m)

m

Output hidden; open in https://colab.research.google.com to view.

#New York

In [77]:
!mkdir ./datasets/ny

##Downloading Relevant Datasets

###NYPD Arrests Data (Historic)

In [ ]:
!wget https://data.cityofnewyork.us/api/views/8h9b-rp9u/rows.csv?accessType=DOWNLOAD -O ./datasets/ny/arr_his.csv

###NYPD Arrest Data 2020

In [ ]:
!wget https://data.cityofnewyork.us/api/views/uip8-fykc/rows.csv?accessType=DOWNLOAD -O ./datasets/ny/arr_2020.csv

###NYPD Shooting Incident Data (Historic)

In [ ]:
!wget https://data.cityofnewyork.us/api/views/833y-fsy8/rows.csv?accessType=DOWNLOAD -O ./datasets/ny/shot_his.csv

###NYPD Shooting Incident Data 2020

In [ ]:
!wget https://data.cityofnewyork.us/api/views/5ucz-vwe8/rows.csv?accessType=DOWNLOAD -O ./datasets/ny/shot_2020.csv

##Datasets' Preparation + Preprocessing

In [ ]:
sf_arr_19 = tc.SFrame.read_csv("./datasets/ny/arr_his.csv")
sf_arr_19

In [ ]:
sf_arr_19['Year'] = sf_arr_19['ARREST_DATE'].apply(lambda x: x.split("/")[2])
sf_arr_19 = sf_arr_19[sf_arr_19['Year'] == '2019']
sf_arr_19

In [ ]:
sf_arr_20 = tc.SFrame.read_csv("./datasets/ny/arr_2020.csv")
sf_arr_20

In [ ]:
sf_arr_20['Year'] = '2020'
sf_arr_20

In [ ]:
sf_arr_19 = sf_arr_19.remove_columns(['ARREST_KEY', 'PD_CD', 'KY_CD', 
                                      'LAW_CODE', 'ARREST_PRECINCT', 'JURISDICTION_CODE', 
                                      'X_COORD_CD', 'Y_COORD_CD', 'Lon_Lat'])
sf_arr_19

In [ ]:
sf_arr_20 = sf_arr_20.remove_columns(['ARREST_KEY', 'PD_CD', 'KY_CD', 
                                      'LAW_CODE', 'ARREST_PRECINCT', 'JURISDICTION_CODE', 
                                      'X_COORD_CD', 'Y_COORD_CD', 
                                      'New Georeferenced Column'])
sf_arr_20

In [ ]:
sf_arr = sf_arr_19.append(sf_arr_20)
sf_arr

In [89]:
sf_arr['PERP_RACE'].unique()

dtype: str
Rows: 7
['BLACK HISPANIC', 'UNKNOWN', 'AMERICAN INDIAN/ALASKAN NATIVE', 'WHITE HISPANIC', 'ASIAN / PACIFIC ISLANDER', 'WHITE', 'BLACK']

In [90]:
borough_dir = {'B': 'Bronx', 'K': 'Brooklyn', 'M': 'Manhattan', 'Q': 'Queens', 'S': 'Staten Island'}

* The following dataset includes details of every arrest, since January 1, 2019. I wanted to focus on arrests of young suspects (under 18), and examine the differences between arrests of black and white suspects.

In [91]:
sf_arr['under_18'] = sf_arr['AGE_GROUP'].apply(lambda x: 1 if (x == '<18') else 0)
sf_arr['b_arr'] = sf_arr['PERP_RACE'].apply(lambda x: 1 if ((x == 'BLACK') or (x == 'BLACK HISPANIC')) else 0)
sf_arr['w_arr'] = sf_arr['PERP_RACE'].apply(lambda x: 1 if ((x == 'WHITE') or (x == 'WHITE HISPANIC')) else 0)
sf_arr['ARREST_BORO'] = sf_arr['ARREST_BORO'].apply(lambda x: borough_dir[x])

sf_arr

ARREST_DATE,PD_DESC,OFNS_DESC,LAW_CAT_CD,ARREST_BORO,AGE_GROUP,PERP_SEX,PERP_RACE
01/08/2019,"ARSON 2,3,4",ARSON,F,Brooklyn,25-44,F,BLACK
02/06/2019,,,F,Manhattan,25-44,M,UNKNOWN
04/05/2019,ESCAPE 3,ESCAPE 3,M,Brooklyn,25-44,M,BLACK
02/05/2019,RAPE 3,RAPE,F,Manhattan,25-44,M,BLACK
01/29/2019,RAPE 1,RAPE,F,Queens,25-44,M,BLACK
01/06/2019,,,F,Manhattan,25-44,M,WHITE
03/02/2019,RAPE 3,RAPE,F,Brooklyn,45-64,M,BLACK
11/27/2019,RAPE 1,RAPE,F,Brooklyn,18-24,M,BLACK HISPANIC
01/14/2019,,,F,Brooklyn,45-64,M,BLACK
01/13/2019,"ARSON 2,3,4",ARSON,F,Manhattan,45-64,F,BLACK


In [ ]:
sf_arr_counts = sf_arr.groupby(key_column_names=['ARREST_DATE'], operations={'tot_count': agg.COUNT(),
                                                                             'under_18_count': agg.SUM('under_18'),
                                                                             'w_count': agg.SUM('w_arr'),
                                                                             'b_count': agg.SUM('b_arr')})
sf_arr_counts['under_18_rate'] = sf_arr_counts.apply(lambda x: x['under_18_count'] / x['tot_count'] * 100)
sf_arr_counts['b_rate'] = sf_arr_counts.apply(lambda x: x['b_count'] / x['tot_count'] * 100)
sf_arr_counts['w_rate'] = sf_arr_counts.apply(lambda x: x['w_count'] / x['tot_count'] * 100)

sf_arr_counts

* Daily counts and rates according to the dataset:

In [93]:
df_arr_counts = sf_arr_counts.to_dataframe()
df_arr_counts['date'] = pd.to_datetime(df_arr_counts['ARREST_DATE'])
df_arr_counts.sort_values(by=['date'], inplace=True, ascending=True)
df_arr_counts

,ARREST_DATE,tot_count,under_18_count,w_count,b_count,under_18_rate,b_rate,w_rate,date
63,01/01/2019,548,18,194,325,3.284672,59.306569,35.401460,2019-01-01
41,01/02/2019,737,38,288,405,5.156038,54.952510,39.077341,2019-01-02
97,01/03/2019,808,30,282,468,3.712871,57.920792,34.900990,2019-01-03
133,01/04/2019,781,34,280,434,4.353393,55.569782,35.851472,2019-01-04
377,01/05/2019,523,13,168,311,2.485660,59.464627,32.122371,2019-01-05
...,...,...,...,...,...,...,...,...,...
329,06/26/2020,212,8,69,122,3.773585,57.547170,32.547170,2020-06-26
171,06/27/2020,175,3,72,88,1.714286,50.285714,41.142857,2020-06-27
533,06/28/2020,188,2,80,87,1.063830,46.276596,42.553191,2020-06-28
466,06/29/2020,228,13,85,121,5.701754,53.070175,37.280702,2020-06-29


In [ ]:
sf_boro_arr_counts = sf_arr.groupby(key_column_names=['ARREST_BORO', 'Year'], operations={'count': agg.COUNT()})

sf_boro_arr_counts

* Yearly number of arrests in each NY borough:

In [95]:
df_boro_arr_counts = sf_boro_arr_counts.to_dataframe()
df_boro_arr_counts

,ARREST_BORO,Year,count
0,Brooklyn,2020,20029
1,Brooklyn,2019,58288
2,Queens,2019,44567
3,Bronx,2019,48815
4,Manhattan,2020,18513
5,Queens,2020,15855
6,Bronx,2020,17158
7,Manhattan,2019,53915
8,Staten Island,2020,3229
9,Staten Island,2019,9032


In [ ]:
sf_shot_19 = tc.SFrame.read_csv("./datasets/ny/shot_his.csv")
sf_shot_19

In [ ]:
sf_shot_19['Year'] = sf_shot_19['OCCUR_DATE'].apply(lambda x: x.split("/")[2])
sf_shot_19 = sf_shot_19[sf_shot_19['Year'] == '2019']
sf_shot_19

In [ ]:
sf_shot_20 = tc.SFrame.read_csv("./datasets/ny/shot_2020.csv")
sf_shot_20

In [ ]:
sf_shot_20 = sf_shot_20.rename({'New Georeferenced Column': 'Lon_Lat'})
sf_shot_20['X_COORD_CD'] = sf_shot_20['X_COORD_CD'].astype(str)
sf_shot_20['Y_COORD_CD'] = sf_shot_20['Y_COORD_CD'].astype(str)
sf_shot_20['Year'] = '2020'
sf_shot_20

In [ ]:
sf_shot = sf_shot_19.append(sf_shot_20)
sf_shot

* The following dataset includes every shooting case reported in NY, since January 1, 2019:

In [102]:
sf_shot['month'] = sf_shot['OCCUR_DATE'].apply(lambda x: x.split('/')[0] + '/' + x.split('/')[2])
sf_shot

INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,PRECINCT,JURISDICTION_CODE,LOCATION_DESC
201575314,08/23/2019,22:10:00,QUEENS,103,0,
205748546,11/27/2019,15:54:00,BRONX,40,0,
193118596,02/02/2019,19:40:00,MANHATTAN,23,0,
204192600,10/24/2019,00:52:00,STATEN ISLAND,121,0,PVT HOUSE
201483468,08/22/2019,18:03:00,BRONX,46,0,
198255460,06/07/2019,17:50:00,BROOKLYN,73,0,
194570529,03/11/2019,16:30:00,BROOKLYN,81,0,
203211777,10/03/2019,01:45:00,BROOKLYN,67,0,MULTI DWELL - APT BUILD
193694863,02/17/2019,03:00:00,QUEENS,114,2,MULTI DWELL - PUBLIC HOUS
199582060,07/10/2019,02:56:00,BROOKLYN,69,0,


In [ ]:
sf_shot_counts = sf_shot.groupby(key_column_names='month', operations={'tot_count': agg.COUNT()})
sf_shot_counts

* Monthly number of shootings in NY:

In [104]:
df_shot_counts = sf_shot_counts.to_dataframe()
df_shot_counts['month'] = pd.to_datetime(df_shot_counts['month'])
df_shot_counts.sort_values(by=['month'], inplace=True, ascending=True)
df_shot_counts

,month,tot_count
12,2019-01-01,63
14,2019-02-01,59
16,2019-03-01,65
17,2019-04-01,67
8,2019-05-01,81
7,2019-06-01,107
5,2019-07-01,123
1,2019-08-01,105
9,2019-09-01,86
11,2019-10-01,80


In [ ]:
sf_boro_shot_counts = sf_shot.groupby(key_column_names=['BORO', 'Year'], operations={'count': agg.COUNT()})

sf_boro_shot_counts

* Yearly number of shootings per borough:

In [106]:
df_boro_shot_counts = sf_boro_shot_counts.to_dataframe()
df_boro_shot_counts

,BORO,Year,count
0,BRONX,2020,173
1,QUEENS,2020,115
2,STATEN ISLAND,2020,21
3,MANHATTAN,2020,106
4,STATEN ISLAND,2019,26
5,MANHATTAN,2019,145
6,BROOKLYN,2020,249
7,BROOKLYN,2019,372
8,BRONX,2019,266
9,QUEENS,2019,158


##Charts

###Daily Charts (2019-Present)

* The New York State's "Stay-at-Home" order started on March 22, and ended gradually starting from May 15.
* In the following chart, we can see a decrease in the number of arrests during this period, and increase afterwards which reached its peak on June 1, during the protests and riots following the killing of George Floyd.
* Affected by this, the NYSP has started a [law-enforcement reform](https://www.governor.ny.gov/news/no-203-new-york-state-police-reform-and-reinvention-collaborative) since then. This probably explains the low number in arrests since June.

In [107]:
fig = px.bar(df_arr_counts, x="date", y="tot_count", 
             color="tot_count",
             title='Daily # Arrests in NY (2019-Present)', 
             labels={"date": "Date",
                     "tot_count": "#"})
fig.show()

* The daily rate of suspects ages under 18 has not changed since 2019.

In [108]:
fig = px.line(df_arr_counts, x='date', y='under_18_rate',
              title='Daily Rate of Arrests of Suspects Aged Under 18 in NY (2019-Present)', 
              labels={'date': 'Date',
                      'under_18_rate': '%'})
fig['data'][0]['line']['color']="#FF00FF"
fig.show()

* The following chart shows the difference of the rate between arrests of black suspects and white suspects.

In [109]:
x = df_arr_counts['date']
y_b = df_arr_counts['b_rate']
y_w = df_arr_counts['w_rate']

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y_b, mode='lines', name='Black'))
fig.add_trace(go.Scatter(x=x, y=y_w, mode='lines', name='White'))

fig.update_layout(title='Daily Rate of Arrests of Suspects in NY (2019-Present)',
                   xaxis_title='Date',
                   yaxis_title='%')
fig.show()

###Monthly Charts (2019-Present)

* The monthly number of shooting cases in NY has increased sharply since the re-opening (~May 2020), and compared to 2019.

In [110]:
fig = px.bar(df_shot_counts, x="month", y="tot_count", 
             color="tot_count",
             title='Monthly # Shootings in NY (2019-Present)', 
             labels={"month": "Month",
                     "tot_count": "#"})
fig.show()

###Yearly Charts (2019 vs. 2020 (As of Today) )

The following two charts show the yearly number of arrests and shooting cases per borough:  

In [111]:
labels_19 = df_boro_arr_counts[df_boro_arr_counts['Year'] == '2019']['ARREST_BORO']
labels_20 = df_boro_arr_counts[df_boro_arr_counts['Year'] == '2020']['ARREST_BORO']

values_19 = df_boro_arr_counts[df_boro_arr_counts['Year'] == '2019']['count']
values_20 = df_boro_arr_counts[df_boro_arr_counts['Year'] == '2020']['count']

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels_19, values=values_19, name="2019"), 1, 1)
fig.add_trace(go.Pie(labels=labels_20, values=values_20, name="2020"), 1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent")

fig.update_layout(
    title_text="Yearly Arrests in NY",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='2019', x=0.195, y=0.5, font_size=20, showarrow=False),
                 dict(text='2020 (As of Today)', x=0.85, y=0.5, font_size=14, showarrow=False)])
fig.show()

In [113]:
labels_19 = df_boro_shot_counts[df_boro_shot_counts['Year'] == '2019']['BORO']
labels_20 = df_boro_shot_counts[df_boro_shot_counts['Year'] == '2020']['BORO']

values_19 = df_boro_shot_counts[df_boro_shot_counts['Year'] == '2019']['count']
values_20 = df_boro_shot_counts[df_boro_shot_counts['Year'] == '2020']['count']

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=labels_19, values=values_19, name="2019"), 1, 1)
fig.add_trace(go.Pie(labels=labels_20, values=values_20, name="2020"), 1, 2)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+percent")

fig.update_layout(
    title_text="Yearly Shootings in NY",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='2019', x=0.195, y=0.5, font_size=20, showarrow=False),
                 dict(text='2020 (As of Today)', x=0.85, y=0.5, font_size=14, showarrow=False)])
fig.show()

#Seattle

In [114]:
!mkdir ./datasets/sea

##Downloading Relevant Datasets

###Call Data

In [ ]:
!wget https://data.seattle.gov/api/views/33kz-ixgy/rows.csv?accessType=DOWNLOAD -O ./datasets/sea/call.csv

###SPD Crime Data: 2008-Present

In [ ]:
!wget https://data.seattle.gov/api/views/tazs-3rd5/rows.csv?accessType=DOWNLOAD -O ./datasets/sea/crime.csv

##Datasets' Preparation + Preprocessing

In [ ]:
sf_call = tc.SFrame.read_csv("./datasets/sea/call.csv")
sf_call

In [ ]:
sf_call['Year'] = sf_call['Original Time Queued'].apply(lambda x: x.split()[0].split("/")[2])
sf_call = sf_call[(sf_call['Call Type'] == '911')]
sf_call = sf_call[(sf_call['Year'] == '2019') | (sf_call['Year'] == '2020')]
sf_call['Date'] = sf_call['Original Time Queued'].apply(lambda x: x.split()[0])
sf_call

In [119]:
sorted(list(sf_call['Final Call Type'].unique()))

['--ALARM-COMM (INC BANK, ATM, SCHOOLS, BSN)',
 '--ALARM-COMM ROBB (BANK, PANIC, DURESS)',
 '--ALARM-OTHER (VARDA,PDT,FIRE,LOCAL,METRO,ETC))',
 '--ALARM-RESIDENTIAL BURG',
 '--ALARM-RESIDENTIAL PANIC OR DURESS',
 '--ALARM-VEHICLE ALARMS',
 '--ANIMAL COMPLAINT - INJURED,DEAD,DANGEROUS',
 '--ANIMAL COMPLAINT - NOISE,STRAY,BITE',
 '--ARSON, BOMBS, EXPLO - ARSON',
 '--ARSON, BOMBS, EXPLO - RECKLESS BURNING',
 '--ARSON,BOMBS,EXPLO - BOMBS, EXPLO,LARGE FIREWORKS',
 '--ASSAULTS - FIREARM INVOLVED',
 '--ASSAULTS - GANG RELATED',
 '--ASSAULTS - HARASSMENT, THREATS',
 '--ASSAULTS - TELEPHONE, WRITING',
 '--ASSAULTS, OTHER',
 '--ASSIST OTHER AGENCY - CITY AGENCY',
 '--ASSIST OTHER AGENCY - COUNTY AGENCY',
 '--ASSIST OTHER AGENCY - FEDERAL AGENCY',
 '--ASSIST OTHER AGENCY - STATE AGENCY',
 '--ASSIST PUBLIC - 911 HANG UP, OPEN LINE',
 '--ASSIST PUBLIC - OTHER (NON-SPECIFIED)',
 '--AUTOMOBILES - ABANDONED CAR',
 '--AUTOMOBILES - AUTO THEFT',
 '--AUTOMOBILES - AUTO THEFT & RECOVERY',
 '--AUTOMOBILES 

* The following datasets includes 911 calls in Seattle, since January 1, 2019. As in the previous cities, I'll focus here on domestic violence related calls: 

In [120]:
sf_call['dom'] = sf_call['Final Call Type'].apply(lambda x: 1 if "--DV" in x else 0)
sf_call

CAD Event Number,Event ClearanceDescription ...,Call Type,Priority,Initial Call Type
2020000207712,PHYSICAL ARREST MADE,911,2,THREATS (INCLS IN-PERSON/BY PHONE/IN ...
2019000292002,UNABLE TO LOCATE INCIDENTOR COMPLAINANT ...,911,2,"SUSPICIOUS PERSON,VEHICLE OR INCIDENT ..."
2019000434821,ASSISTANCE RENDERED,911,2,"DISTURBANCE,MISCELLANEOUS/OTHER ..."
2020000044531,UNABLE TO LOCATE INCIDENTOR COMPLAINANT ...,911,3,NUISANCE - MISCHIEF
2020000135892,REPORT WRITTEN (NOARREST) ...,911,1,"WEAPN-IP/JO-GUN,DEADLYWPN (NO THRT/ASLT/DIST) ..."
2019000476371,ASSISTANCE RENDERED,911,4,"NOISE - DIST, GENERAL(CONST, RESID, BALL P ..."
2019000088157,ASSISTANCE RENDERED,911,3,DETOX - REQUEST FOR
2019000135408,DUPLICATED OR CANCELLEDBY RADIO ...,911,1,DIST - IP/JO - DV DIST -NO ASLT ...
2019000428472,UNABLE TO LOCATE INCIDENTOR COMPLAINANT ...,911,2,MVC - UNK INJURIES
2019000265135,UNABLE TO LOCATE INCIDENTOR COMPLAINANT ...,911,1,ASLT - IP/JO - WITH ORW/O WPNS (NO SHOOTINGS) ...


In [ ]:
sf_call_counts = sf_call.groupby(key_column_names='Date', operations={'tot_count': agg.COUNT(),
                                                                      'dom_count': agg.SUM('dom')})
sf_call_counts['dom_rate'] = sf_call_counts.apply(lambda x: x['dom_count'] / x['tot_count'] * 100)

sf_call_counts

* Daily counts and rate of domestic violence related 911 calls:

In [122]:
df_call_counts = sf_call_counts.to_dataframe()
df_call_counts['date'] = pd.to_datetime(df_call_counts['Date'])
df_call_counts.sort_values(by=['date'], inplace=True, ascending=True)
df_call_counts

,Date,tot_count,dom_count,dom_rate,date
73,01/01/2019,414,33,7.971014,2019-01-01
48,01/02/2019,354,13,3.672316,2019-01-02
114,01/03/2019,417,11,2.637890,2019-01-03
151,01/04/2019,379,16,4.221636,2019-01-04
418,01/05/2019,379,18,4.749340,2019-01-05
...,...,...,...,...,...
579,08/21/2020,385,18,4.675325,2020-08-21
488,08/22/2020,348,19,5.459770,2020-08-22
345,08/23/2020,339,17,5.014749,2020-08-23
62,08/24/2020,370,10,2.702703,2020-08-24


In [ ]:
sf_crime = tc.SFrame.read_csv("./datasets/sea/crime.csv")
sf_crime

In [ ]:
sf_crime['Year'] = sf_crime['Offense Start DateTime'].apply(lambda x: x.split("-")[0])
sf_crime = sf_crime[(sf_crime['Year'] == '2019') | (sf_crime['Year'] == '2020')]
sf_crime['Date'] = sf_crime['Offense Start DateTime'].apply(lambda x: x.split()[0])
sf_crime

In [131]:
sorted(list(sf_crime['Crime Against Category'].unique()))

['NOT_A_CRIME', 'PERSON', 'PROPERTY', 'SOCIETY']

In [ ]:
sf_crime = sf_crime[sf_crime['Crime Against Category'] != 'NOT_A_CRIME']
sf_crime

In [133]:
sorted(list(sf_crime['Offense Parent Group'].unique()))

['ANIMAL CRUELTY',
 'ARSON',
 'ASSAULT OFFENSES',
 'BAD CHECKS',
 'BRIBERY',
 'BURGLARY/BREAKING&ENTERING',
 'COUNTERFEITING/FORGERY',
 'CURFEW/LOITERING/VAGRANCY VIOLATIONS',
 'DESTRUCTION/DAMAGE/VANDALISM OF PROPERTY',
 'DRIVING UNDER THE INFLUENCE',
 'DRUG/NARCOTIC OFFENSES',
 'DRUNKENNESS',
 'EMBEZZLEMENT',
 'EXTORTION/BLACKMAIL',
 'FAMILY OFFENSES, NONVIOLENT',
 'FRAUD OFFENSES',
 'HOMICIDE OFFENSES',
 'HUMAN TRAFFICKING',
 'KIDNAPPING/ABDUCTION',
 'LARCENY-THEFT',
 'LIQUOR LAW VIOLATIONS',
 'MOTOR VEHICLE THEFT',
 'PEEPING TOM',
 'PORNOGRAPHY/OBSCENE MATERIAL',
 'PROSTITUTION OFFENSES',
 'ROBBERY',
 'SEX OFFENSES',
 'SEX OFFENSES, CONSENSUAL',
 'STOLEN PROPERTY OFFENSES',
 'TRESPASS OF REAL PROPERTY',
 'WEAPON LAW VIOLATIONS']

* The following dataset includes reported crime cases in Seattle, since January 1, 2019. I'll focus on: drugs cases, cruelty to animals, and murders.

In [134]:
sf_crime['cruelty_to_animal'] = sf_crime['Offense Parent Group'].apply(lambda x: 1 if (x =='ANIMAL CRUELTY') else 0)
sf_crime['drugs'] = sf_crime['Offense Parent Group'].apply(lambda x: 1 if (x =='DRUG/NARCOTIC OFFENSES') else 0)
sf_crime['murder'] = sf_crime['Offense Parent Group'].apply(lambda x: 1 if (x =='HOMICIDE OFFENSES') else 0)

sf_crime

Report Number,Offense ID,Offense Start DateTime,Offense End DateTime,Report DateTime,Group A B
2020-031093,12248131760,2020-01-26 01:28:54,,2020-01-26 03:23:07,B
2019-376606,10760841671,2019-10-05 12:00:00,2019-10-09 08:00:00,2019-10-09 09:59:25,A
2019-912779,10517684672,2019-09-24 08:00:00,2019-09-25 21:00:00,2019-09-28 10:11:08,A
2019-347374,10372671894,2019-09-17 00:15:00,2019-09-17 00:10:00,2019-09-17 01:21:11,A
2020-044316,12605323404,2020-02-05 06:51:49,,2020-02-05 06:51:57,B
2020-044112,12605040083,2020-02-04 21:45:00,2020-02-04 23:00:00,2020-02-04 23:45:19,B
2020-043564,12604657915,2020-02-04 14:10:00,,2020-02-04 14:47:51,B
2020-043255,12604780082,2020-02-04 10:54:00,2020-02-04 10:58:00,2020-02-04 14:28:52,B
2020-043146,12604484012,2020-02-04 09:25:00,,2020-02-04 14:11:33,B
2020-043199,12604324385,2020-02-04 10:07:00,2020-02-04 19:00:00,2020-02-04 13:12:41,B


In [ ]:
sf_crime_counts = sf_crime.groupby(key_column_names='Date', operations={'tot_count': agg.COUNT(), 
                                                                        'animal_count': agg.SUM('cruelty_to_animal'),
                                                                        'drugs_count': agg.SUM('drugs'),
                                                                        'murder_count': agg.SUM('murder')})


sf_crime_counts['drugs_rate'] = sf_crime_counts.apply(lambda x: x['drugs_count'] / x['tot_count'] * 100)
sf_crime_counts['monthstring'] = sf_crime_counts.apply(lambda x: x['Date'].split('-')[1] + '/' + x['Date'].split('-')[0])

sf_crime_counts

In [ ]:
sf_crime_counts_month = sf_crime_counts.groupby(key_column_names='monthstring', operations={'tot_count': agg.SUM('tot_count'),
                                                                                            'animal_count': agg.SUM('animal_count'),                        
                                                                                            'murder_count': agg.SUM('murder_count')})
sf_crime_counts_month['animal_rate'] = sf_crime_counts_month.apply(lambda x: x['animal_count'] / x['tot_count'] * 100)
sf_crime_counts_month['murder_rate'] = sf_crime_counts_month.apply(lambda x: x['murder_count'] / x['tot_count'] * 100)

sf_crime_counts_month

* Daily counts and drug cases' rate:

In [138]:
df_crime_counts = sf_crime_counts.to_dataframe()
df_crime_counts['date'] = pd.to_datetime(df_crime_counts['Date'])
df_crime_counts.sort_values(by=['date'], inplace=True, ascending=True)
df_crime_counts

,Date,tot_count,animal_count,drugs_count,murder_count,drugs_rate,monthstring,date
423,2019-01-01,279,0,5,0,1.792115,01/2019,2019-01-01
362,2019-01-02,179,0,15,0,8.379888,01/2019,2019-01-02
22,2019-01-03,195,0,9,0,4.615385,01/2019,2019-01-03
435,2019-01-04,197,0,6,0,3.045685,01/2019,2019-01-04
128,2019-01-05,155,0,4,0,2.580645,01/2019,2019-01-05
...,...,...,...,...,...,...,...,...
511,2020-08-23,84,0,2,0,2.380952,08/2020,2020-08-23
250,2020-08-24,86,0,3,0,3.488372,08/2020,2020-08-24
92,2020-08-25,90,0,0,0,0.000000,08/2020,2020-08-25
549,2020-08-26,81,0,0,0,0.000000,08/2020,2020-08-26


* Monthly counts and rates of: cruelty against animals, and murders:

In [139]:
df_crime_counts_month = sf_crime_counts_month.to_dataframe()
df_crime_counts_month['month'] = pd.to_datetime(df_crime_counts_month['monthstring'])
df_crime_counts_month.sort_values(by=['month'], inplace=True, ascending=True)
df_crime_counts_month

,monthstring,tot_count,animal_count,murder_count,animal_rate,murder_rate,month
12,01/2019,6294,0,3,0.000000,0.047664,2019-01-01
14,02/2019,4669,0,1,0.000000,0.021418,2019-02-01
18,03/2019,5800,0,2,0.000000,0.034483,2019-03-01
19,04/2019,5815,0,1,0.000000,0.017197,2019-04-01
8,05/2019,5768,2,4,0.034674,0.069348,2019-05-01
7,06/2019,5659,2,1,0.035342,0.017671,2019-06-01
5,07/2019,6298,3,6,0.047634,0.095268,2019-07-01
1,08/2019,6341,6,5,0.094622,0.078852,2019-08-01
9,09/2019,5840,0,1,0.000000,0.017123,2019-09-01
11,10/2019,6032,1,1,0.016578,0.016578,2019-10-01


##Charts

###Daily Charts (2019-Present)

* The Washington State's "Stay-at-Home" order started on March 23, and ended gradually in May.
* In the following chart, since the end of March 2020, we can see some increase in the rate of domestic violence 911 calls, and compared to 2019.
* Note: the anomality of November 14, 2019 can be explained by relatively low numbers of 911 calls (21) as reported for this date in the dataset (see code beneath the chart).

In [140]:
fig = px.line(df_call_counts, x='date', y='dom_rate',
              title='Daily Rate of Domestic Violence\'s 911 Calls in Seattle (2019-Present)', 
              labels={'date': 'Date',
                      'dom_rate': '%'})
fig['data'][0]['line']['color']="#800080"
fig.show()

In [174]:
sf_call[sf_call['Date'] == '11/14/2019'].num_rows()

21

In [175]:
sf_call[sf_call['Date'] == '11/13/2019'].num_rows()

106

In [176]:
sf_call[sf_call['Date'] == '11/15/2019'].num_rows()

399

* The following chart displays the daily number of crime cases in Seattle, since January 1, 2019.
* The average number of crimes has not changed during the "Stay-at-Home" order period, and even increased dramatically until the mid May.
* In the middle of may, a [widespread unemployment](https://spdblotter.seattle.gov/2020/05/08/criminals-exploiting-covid-19-to-commit-unemployment-fraud/) fraud had been discovered. And indeed, the unusual large number of crime cases around the middle of may are of "Identity Theft" as shown in the code below the chart. 

In [148]:
fig = px.bar(df_crime_counts, x="date", y="tot_count",
             title='Daily # Crimes in Seattle (2019-Present)',
             color = "tot_count",
             labels={"date": "Date",
                     "tot_count": "#"})
fig.show()

In [179]:
sf_t = sf_crime[sf_crime['Date'] == "2020-05-18"]
df_t = sf_t.to_dataframe()
df_t['Offense'].value_counts()[:3]

Identity Theft                  585
Theft From Motor Vehicle         32
Burglary/Breaking & Entering     32
Name: Offense, dtype: int64

In [180]:
sf_t = sf_crime[sf_crime['Date'] == "2020-05-15"]
df_t = sf_t.to_dataframe()
df_t['Offense'].value_counts()[:3]

Identity Theft                  500
Theft From Motor Vehicle         29
Burglary/Breaking & Entering     28
Name: Offense, dtype: int64

* The daily rate of drug offenses has decreased, and lower compared to 2019.

In [142]:
fig = px.line(df_crime_counts, x='date', y='drugs_rate',
              title='Daily Rate of Drugs\' Crimes in Seattle (2019-Present)', 
              labels={'date': 'Date',
                      'drugs_rate': '%'})
fig['data'][0]['line']['color']="#800000"
fig.show()

###Monthly Charts (2019-Present)

The following chart shows that since the end of March, the monthly rate of murders has increased (and higher compared to 2019).

In [143]:
fig = px.bar(df_crime_counts_month, x="month", y="murder_rate",
             color='murder_rate',
             title='Monthly Rate of Murders in Seattle (2019-Present)', 
             labels={"month": "Month",
                     "murder_rate": "%"})
fig.show()

* The monthly rate of cruelty against animals has increased since the re-opening.

In [144]:
fig = px.bar(df_crime_counts_month, x="month", y="animal_rate",
             color='animal_rate',
             title='Monthly Rate of Cruelty Against Animals in LA (2019-Present)', 
             labels={"month": "Month",
                     "animal_rate": "%"})
fig.show()

##Daily Crime Map (2019-Present)

* An Interactive map of every crime reported in Seattle since January 1, 2019 (it displays, for example, the sharp increase in the number of crimes on May 18, 2020).

In [145]:
days = list(df_crime_counts['Date'])
data_cd = []
for day in days:
  l_temp = []
  sf_temp = sf_crime[sf_crime['Date'] == day]
  for row in range(sf_temp.num_rows()):
    l_temp.append([sf_temp['Latitude'][row], sf_temp['Longitude'][row]])
  data_cd.append(l_temp)

In [ ]:
m = folium.Map(loc['sea'], zoom_start=11)
m

In [147]:
hm = plugins.HeatMapWithTime(
    data=data_cd,
    index=days,
    auto_play=True,
)

hm.add_to(m)

m

Output hidden; open in https://colab.research.google.com to view.

#Summary/ Conclusions:

Based on the data of these cities, in overall we can say that:
* The number of murders/ shootings has increased since the re-openings, although it seems like the total number of crimes has decreased (compared to 2019).
* Also, there is an increase in the domestic violence cases (based on reported crimes, and calls-for-service), as I assumed at first.
* Probably the drugs offenses rate has decreased following COVID-19 measures and social limitations. 